# Consultas centradas no termo

Quando os campos vêm diretamente do modelo de dados de origem, os cálculos TF × IDF que resultam das pesquisas não se alinham perfeitamente às expectativas do usuário. TF × IDF pode ser uma métrica ruim quando o campo (e seus scores) não são mapeados para signals que os usuários se preocupam no processo de ranqueamento. Quando você pesquisa em dezenas de campos, simplesmente porque "é isso que está no banco de dados", muitas anomalias de pontuação podem aparecer que acabam por não corresponder às expectativas do usuário.

A pesquisa centrada em campo amplifica a discordância de signal. Ao calcular o score de cada campo isoladamente, a pesquisa centrada em campo é propensa a influenciar fortemente os resultados da pesquisa em uma direção ou outra. Ao executar pesquisas centradas em campo em dezenas e dezenas de campos (todos os critérios possíveis do seu banco de dados), busca-se problemas.

O modelo de dados de origem mantém você preso em uma visualização bottom-up, modelo de dados de origem primeiro.
Para uma boa modelagem de signals, é interessante pensar top-down e primeiramente no usuário. Com o que os usuários se preocupam ao classificar? Como você pode criar campos, a partir do modelo de dados de origem, para calcular esses signals? 

A pesquisa centrada no termo pode ajudar a fornecer essa perspectiva top-down na modelagem de signals.

A pesquisa centrada no termo resolve o problema do elefante albino e discordância de signals ao adotar uma perspectiva top-down da pesquisa. Mas também apresenta problemas que veremos ao longo das análises. No fim das contas, uma abordagem mista pode ser a solução.

Neste notebook, vamos realizar pesquisas centradas no termo no Solr. 

Vamos ver abordagens de pesquisa centrada no termo e demonstrar seus problemas.

As diversas possibilidades de pesquisa no Solr podem ser encontradas em:

https://lucene.apache.org/solr/guide/8_0/query-syntax-and-parsing.html

Pré-requisitos:
- Ter executado o notebook Parte II/Indexação dos dados: 
    No fim deste notebook, realiza-se a indexação dos dados do TMDB com um novo pipeline de análise baseado no português

# Inicialização

In [1]:
#PARÂMETROS

#Máquina e porta (formato host:port)
SOLR_ADDR='localhost:8983'
ELASTIC_ADDR='localhost:9200'

In [2]:
import json
import requests
import pandas as pd
from datetime import datetime
headers = {'content-type': 'application/json;charset=UTF-8'}

def date_diff_in_seconds(dt2, dt1):
    timedelta = dt2 - dt1
    return timedelta.days * 24 * 3600 + timedelta.seconds

# Some utilities for flattening the explain into something a bit more
# readable. Pass Explain JSON, get readable (ironically this is what Solr's default output is :-p)
def flatten(l):
    [item for sublist in l for item in sublist]

def simplerExplain(explainJson, depth=0):
    result = " " * (depth * 2) + "%s, %s\n" % (explainJson['value'], explainJson['description'])
    #print json.dumps(explainJson, indent=True)
    if 'details' in explainJson:
        for detail in explainJson['details']:
            result += simplerExplain(detail, depth=depth+1)
    return result

In [3]:
import pickle

#Retorna um dicionário do python com os dados dos filmes
def extract():
    return pickle.load(open("../Dados/movies.p","rb"))

In [4]:
#Cria um índice novo no Solr e reindexa os dados
def reindex_solr(movieDict={}, delete=True):
    if delete:
        resp = requests.get("http://" + SOLR_ADDR + "/solr/admin/collections?action=DELETE&name=tmdb")
        resp = requests.get("http://" + SOLR_ADDR + "/solr/admin/collections?action=CREATE&name=tmdb&numShards=1")
        print("solr building...", resp.status_code)
    
    movies = ""
    
    for id, movie in movieDict.items():
        movies += json.dumps(movie) + ","
    
    bulkMovies = "[" + movies + "]"

    print("solr indexing...")
    resp = requests.post("http://" + SOLR_ADDR + "/solr/tmdb/update/json/docs?commit=true", data=bulkMovies, headers=headers)
    print("solr indexing done.", resp.status_code, resp.text)

In [5]:
#Faz a pesquisa especificada no Solr e imprime os resultados 
def search_solr(usersSearch, qf='title overview'):
    url = 'http://' + SOLR_ADDR + '/solr/tmdb/select?q='+ usersSearch + '&defType=edismax&qf=' + qf + '&rows=30&wt=json&fl=title,overview,cast.name, directors.name,score'
    httpResp = requests.get(url, headers=headers) #A
    searchHits = json.loads(httpResp.text)['response']['docs']
    print("Solr results")
    lista_resultados = []
    for idx, hit in enumerate(searchHits):
        filme = [idx + 1, hit['score'], hit['title'], hit['overview'], hit['cast.name'], hit['directors.name']]
        lista_resultados.append(filme)
    
    pd.set_option('display.max_colwidth', -1)
    
    df = pd.DataFrame(lista_resultados,columns=['Num', 'Relevance Score', 'Movie Title', 'Overview', 'Cast', 'Director'], index=None)
    return df

In [6]:
#Faz a pesquisa especificada no Elasticsearch e imprime os resultados
def search_elastic(usersSearch, query=None):
    if not query:
        query = {
            'query': {
                'multi_match': { 
                    'query': usersSearch, #A
                    'fields': ['title^10', 'overview'] #B
                }
            },
            'size': '30'
        }
    
    url = 'http://'+ ELASTIC_ADDR +'/tmdb/_search'
    httpResp = requests.get(url, data=json.dumps(query), headers=headers) #A
    searchHits = json.loads(httpResp.text)['hits']
    print("Elasticsearch results")
    lista_resultados = []
    for idx, hit in enumerate(searchHits['hits']):
        filme = [idx + 1, hit['_score'], hit['_source']['title'], hit['_source']['overview'], get_cast_list(hit['_source']['cast'],10), get_cast_list(hit['_source']['directors'],5)]
        lista_resultados.append(filme)
    
    pd.set_option('display.max_colwidth', -1)
    
    df = pd.DataFrame(lista_resultados,columns=['Num', 'Relevance Score', 'Movie Title', 'Overview', 'Cast', 'Director'], index=None)
    return df

In [7]:
def explain_solr(users_search):
    url = 'http://' + SOLR_ADDR + '/solr/tmdb/select?q='+ users_search + '&debugQuery=true&defType=edismax&qf=title overview cast.name.bigramed directors.name.bigramed&rows=1&wt=json&fl=title,score'
    httpResp = requests.get(url, headers=headers) #A
    explain = json.loads(httpResp.text)['debug']['parsedquery']
    print('Explicação da query no Solr:')
    print(explain)
    print('\n')

In [8]:
def explain_elastic(query):
    httpResp = requests.get('http://'+ ELASTIC_ADDR +'/tmdb/_validate/query?explain',data=json.dumps(query), headers=headers)
    print('Explicação da query no Elasticsearch:')
    json_str= json.dumps(json.loads(httpResp.text), indent=2, ensure_ascii=False).encode('utf-8')
    print(json_str.decode())
    print('\n')

Vamos inicialmente alterar o schema do Solr com a mesma alteração que fizemos no Elasticsearch: criamos dois campos com bigramas - cast.name.bigramed, directors.name.bigramed

In [9]:
#Criação de um novo tipo 
url = 'http://' + SOLR_ADDR + '/solr/tmdb/schema'
data = '"add-field-type" : { \
     "name":"text_pt_bigrammed", \
     "class":"solr.TextField", \
     "positionIncrementGap":"100", \
     "analyzer" : {\
        "tokenizer":{\
           "class":"solr.StandardTokenizerFactory" },\
        "filters":[\
            {"class":"solr.LowerCaseFilterFactory"},\
            {"class":"solr.StopFilterFactory", "format":"snowball" "words":"lang/stopwords_pt.txt" ignoreCase:"true"},\
            {"class":"solr.PortugueseLightStemFilterFactory"},\
            {"class":"solr.ShingleFilterFactory"}\
            ]}}'
httpResp = requests.post(url, data=data,headers=headers)

if httpResp.status_code == 400:    
    print('O tipo de campo já está criado!')
    print(httpResp.text)
elif httpResp.status_code == 200:    
    print('Campo criado com sucesso!')

O tipo de campo já está criado!
{
  "responseHeader":{
    "status":400,
    "QTime":124},
  "error":{
    "metadata":[
      "error-class","org.apache.solr.api.ApiBag$ExceptionWithErrObject",
      "root-error-class","org.apache.solr.api.ApiBag$ExceptionWithErrObject"],
    "details":[{
        "add-field-type":{
          "name":"text_pt_bigrammed",
          "class":"solr.TextField",
          "positionIncrementGap":"100",
          "analyzer":{
            "tokenizer":{"class":"solr.StandardTokenizerFactory"},
            "filters":[{"class":"solr.LowerCaseFilterFactory"},
              {
                "class":"solr.StopFilterFactory",
                "format":"snowball",
                "words":"lang/stopwords_pt.txt",
                "ignoreCase":"true"},
              {"class":"solr.PortugueseLightStemFilterFactory"},
              {"class":"solr.ShingleFilterFactory"}]}},
        "errorMessages":["Field type 'text_pt_bigrammed' already exists.\n"]}],
    "msg":"error processi

In [10]:
#Tenta criar o novo campo cast.name.bigramed. Caso dê erro, tenta fazer um replace.
data = '"add-field":{\
         "name":"cast.name.bigramed",\
         "type":"text_pt_bigrammed",\
         "stored":true,\
         "multiValued":true}'
httpResp = requests.post(url, data=data,headers=headers)

if httpResp.status_code == 400:
    data = '"replace-field":{\
         "name":"cast.name.bigramed",\
         "type":"text_pt_bigrammed",\
         "stored":true,\
         "multiValued":true}'
    httpResp = requests.post(url, data=data,headers=headers)
    
print('Campo criado! status code:', httpResp.status_code)
print(httpResp.text)

Campo criado! status code: 200
{
  "responseHeader":{
    "status":0,
    "QTime":466}}



In [14]:
#Tenta criar o novo campo directors.name.bigramed. Caso dê erro, tenta fazer um replace.
data = '"add-field":{\
         "name":"directors.name.bigramed",\
         "type":"text_pt_bigrammed",\
         "stored":true, \
         "multiValued": true}'
httpResp = requests.post(url, data=data,headers=headers)

if httpResp.status_code == 400:
    data = '"replace-field":{\
         "name":"directors.name.bigramed",\
         "type":"text_pt_bigrammed",\
         "stored":true, \
         "multiValued": true}'
    httpResp = requests.post(url, data=data,headers=headers)

print('Campo criado! status code:', httpResp.status_code)
print(httpResp.text)

Campo criado! status code: 200
{
  "responseHeader":{
    "status":0,
    "QTime":508}}



In [15]:
#Cria regra copy-field para os novos campos criados
data = '"add-copy-field":{\
         "source":"cast.name",\
         "dest":[ "cast.name.bigramed"]}'
httpResp = requests.post(url, data=data,headers=headers)
print('Copy field para cast.name.bigramed criado: status code:',httpResp.status_code)
print(httpResp.text)

data = '"add-copy-field":{\
         "source":"directors.name",\
         "dest":[ "directors.name.bigramed"]}'
httpResp = requests.post(url, data=data,headers=headers)
print('Copy field para directors.name.bigramed criado: status code:',httpResp.status_code)
print(httpResp.text)

Copy field para cast.name.bigramed criado: status code: 200
{
  "responseHeader":{
    "status":0,
    "QTime":468}}

Copy field para directors.name.bigramed criado: status code: 200
{
  "responseHeader":{
    "status":0,
    "QTime":990}}



In [16]:
#Reindexqa os dados conforme novo schema
movieDict = extract()
reindex_solr(movieDict)

solr building... 200
solr indexing...
solr indexing done. 200 {
  "responseHeader":{
    "rf":1,
    "status":0,
    "QTime":13310}}



# Vamos começar!



#### 1. consulta centrada no termo nos campos title, overview, cast.name.bigramed e directors.name.bigramed
Vamos executar uma query centrada no termo no Solr e no Elasticsearch

In [17]:
usersSearch = 'jornada nas estrelas patrick stewart william shatner'

df = search_solr(usersSearch, qf='title overview cast.name.bigramed directors.name.bigramed')
df_resumido = df[['Relevance Score', 'Movie Title']]
df_resumido.head(15)

Solr results


,Relevance Score,Movie Title
0,5.525932,Jornada nas Estrelas: O Filme
1,5.033387,X-Men: O Confronto Final
2,4.767649,Jornada nas Estrelas: Generations
3,4.767649,Jornada nas Estrelas: Insurreição
4,4.767649,Jornada nas Estrelas: Nêmesis
5,4.727496,Máquina Quase Mortífera
6,4.582678,Acima das Nuvens
7,4.312117,Logan
8,4.284116,Gangues de Nova York
9,4.263632,Stardust - O Mistério da Estrela


In [19]:
df.head(15)

,Num,Relevance Score,Movie Title,Overview,Cast,Director
0,1,5.525932,Jornada nas Estrelas: O Filme,"Um fenômeno alienígena de poder e tamanho descomunal se aproxima da Terra, destruindo tudo em seu caminho. A única nave que pode enfrentar esta força letal que ameaça a Terra é a U.S.S. Enterprise. O lendário comandante James T. Kirk (William Shatner) é convocado para a missão, mas um perigo não previsto pode destruir a U.S.S. Enterprise e toda a sua tripulação.","[William Shatner, Leonard Nimoy, DeForest Kelley, Stephen Collins, Persis Khambatta, James Doohan, Nichelle Nichols, George Takei, Walter Koenig, Majel Barrett, Grace Lee Whitney, Mark Lenard, Billy Van Zandt, Roger Aaron Brown, Gary Faga, Franklyn Seales]",[Robert Wise]
1,2,5.033387,X-Men: O Confronto Final,"É descoberta uma cura para os mutantes, que agora podem optar por manter seus poderes ou se tornarem seres humanos normais. A descoberta põe em campos opostos Magneto (Ian McKellen), que acredita que esta cura se tornará uma arma contra os mutantes, e os X-Men, liderados pelo professor Charles Xavier (Patrick Stewart).","[Hugh Jackman, Halle Berry, Ian McKellen, Patrick Stewart, Famke Janssen, Anna Paquin, Kelsey Grammer, James Marsden, Rebecca Romijn, Shawn Ashmore, Aaron Stanford, Vinnie Jones, Ellen Page, Daniel Cudmore, Ben Foster, Michael Murphy, Dania Ramirez, Shohreh Aghdashloo, Josef Sommer, Bill Duke, Eric Dane, Desiree Zurowski, Adrian Hough, Haley Ramm, Chris Claremont, Stan Lee, Cayden Boyd, Tanya Newbould, Anthony Heald, Cameron Bright, Connor Widdows, Kea Wong, Shauna Kain, Luke Pohl, Julian Richings, Lloyd Adams, Richard Yee, Via Saleaumua, Meiling Melançon, Omahyra Mota, Clayton Watmough, Ken Leung, Julian Christopher, Brad Kelly, Makenzie Vega Norfolk, Mi-Jung Lee, Benita Ha, Ron James, R. Lee Ermey, Lance Gibson, Aaron Pearl, Chelah Horsdal, John Pyper-Ferguson, Brenna O'Brien, Justin Callan, Alex Ferris, Peter Kawasaki, Ron Blecker, Emy Aneke, David Smith, Olivia Williams]",[Brett Ratner]
2,3,4.767649,Jornada nas Estrelas: Generations,"A viagem inaugural da terceira nave Enterprise está ameaçada pelo cientista louco Soren e apenas uma pessoa pode detê-lo, mas ela está morta há 78 anos.","[Patrick Stewart, Jonathan Frakes, Brent Spiner, LeVar Burton, Michael Dorn, Gates McFadden, Marina Sirtis, William Shatner, James Doohan, Walter Koenig, Malcolm McDowell, Alan Ruck, Whoopi Goldberg, Thomas Dekker, Cameron Oppenheimer, Jenette Goldstein, Tim Russ]",[David Carson]
3,4,4.767649,Jornada nas Estrelas: Insurreição,"No planeta Ba'ku, com apenas 600 habitantes, uma estação arqueológica da Federação em conjunto com a raça So'na é atacada pelo tenente-comandante Data, enviado para observações. Mandado para investigar, Picard recebe a ordem de atacar o planeta, mas ele se rebela. O motivo do ataque: os So'na e a Federação querem a fonte da juventude que existe em Ba'ku.","[Patrick Stewart, Jonathan Frakes, Brent Spiner, LeVar Burton, Michael Dorn, Gates McFadden, Marina Sirtis, Gregg Henry, F. Murray Abraham, Anthony Zerbe, Donna Murphy, Stephanie Niznik, Rico Bueno, Daniel Hugh Kelly, Michael Welch, Mark Deakins, Michael Horton, Bruce French, Breon Gorman, John Hostetter, Rick Worthy, Larry Anderson, D. Elliot Woods, Jennifer Tung, Raye Birk, Peggy Miley, Claudette Nevins, Greg Poland, Kenneth Lane Edwards, Joseph Ruskin, Zachary Isaiah Williams, McKenzie Westmore, Phillip Glasser]",[Jonathan Frakes]
4,5,4.767649,Jornada nas Estrelas: Nêmesis,"O capitão Jean-Luc Picard e a tripulação da Enterprise são enviados em uma missão diplomática de paz rumo ao planeta Romulus. Lá eles precisam deter uma rebelião liderada por Shinzou que ameaça destruir a Terra, bem como enfrentar um perigoso clone do próprio capitão Picard.","[Patrick Stewart, Jonathan Frakes, Brent Spiner, LeVar Burton, Michael Dorn, Gates McFadden, Marina Sirtis, Tom Hardy, Ron Perlman, Shannon Cochran, Dina Meyer, Kate Mulgrew, Wil Wheaton, Whoopi Goldberg, Stuart Baird, Alan Dale, Bryan Singer

Analisando os resultados, percebemos que ainda não conseguimos os filmes em que os dois atuaram juntos... Porque?

No primeiro resultado - Jornada nas Estrelas: O Filme - somente William Shatner atuou. 

O segundo resultado, nem Jornada nas Estrelas é.

O que ocorreu? Vamos ver que query foi executada.

In [20]:
explain_solr(usersSearch)

Explicação da query no Solr:
+DisjunctionMaxQuery(((overview:jornad overview:estrel overview:patrick overview:stewart overview:william overview:shatner) | (((+directors.name.bigramed:jornad +directors.name.bigramed:_ estrel +directors.name.bigramed:patrick +directors.name.bigramed:stewart +directors.name.bigramed:william +directors.name.bigramed:shatner) (+directors.name.bigramed:jornad +directors.name.bigramed:_ estrel +directors.name.bigramed:patrick +directors.name.bigramed:stewart +directors.name.bigramed:william shatner) (+directors.name.bigramed:jornad +directors.name.bigramed:_ estrel +directors.name.bigramed:patrick +directors.name.bigramed:stewart william +directors.name.bigramed:shatner) (+directors.name.bigramed:jornad +directors.name.bigramed:_ estrel +directors.name.bigramed:patrick stewart +directors.name.bigramed:william +directors.name.bigramed:shatner) (+directors.name.bigramed:jornad +directors.name.bigramed:_ estrel +directors.name.bigramed:patrick stewart +directors

### Que maluquice é essa?
![title](img/fascinating.gif)

Vamos ver o que ocorre no Elasticsearch.

Obs: Entendo que o tipo de query query_string deveria inferir o operador OR como default. Mas se eu não coloco operadores, ele gera uma query centrada no campo...

In [22]:
def get_cast_list(cast, n):
    i = 0
    cast_list = []
    for e in cast:
        i += 1
        cast_list.append(e['name'])
    return cast_list

In [25]:
usersSearchElastic = 'jornada OR nas OR estrelas OR patrick OR stewart OR william OR shatner'
query = {
    'query': {
        'query_string': { 
            'query': usersSearchElastic,  #User's query
            'fields': ['title', 'overview', 'cast.name.bigramed', 'directors.name.bigramed'],  
            'minimum_should_match':'30%'
         }
    },
    'size': 50,
    'explain': True
}

df = search_elastic(usersSearch, query)
df_resumido = df[['Relevance Score', 'Movie Title']]
df_resumido.head(15)

Elasticsearch results


,Relevance Score,Movie Title
0,22.645878,Jornada nas Estrelas: O Filme
1,11.073451,X-Men: O Confronto Final
2,10.488827,Jornada nas Estrelas: Generations
3,10.488827,Jornada nas Estrelas: Insurreição
4,10.488827,Jornada nas Estrelas: Nêmesis
5,10.400492,Máquina Quase Mortífera
6,10.081892,Acima das Nuvens
7,9.486658,Logan
8,9.425055,Gangues de Nova York
9,9.379991,Stardust - O Mistério da Estrela


In [26]:
query = {
    'query': {
        'query_string': { 
            'query': usersSearchElastic,  #User's query
            'fields': ['title', 'overview', 'cast.name.bigramed', 'directors.name.bigramed'],  
            'minimum_should_match':'30%'
         }
    },
}
explain_elastic(query)

Explicação da query no Elasticsearch:
{
  "_shards": {
    "total": 1,
    "successful": 1,
    "failed": 0
  },
  "valid": true,
  "explanations": [
    {
      "index": "tmdb",
      "valid": true,
      "explanation": "((overview:jornad | title:jornad) (overview:estrel | title:estrel) (overview:patrick | title:patrick) (overview:stewart | title:stewart) (overview:william | title:william) (overview:shatner | title:shatner))~1"
    }
  ]
}




Alguns campos não estão sendo considerados: cast.name.bigramed e directors.name.bigramed.

Os campos <i>bigramed</i> consideram dois termos juntos para a consulta.

Então, para estes campos, a unidade de pesquisa seria <b>cast.name.bigramed:william shatner</b> e não <b>cast.name.bigramed:william</b> e <b>cast.name.bigramed:shatner</b>.


### Explicação

A pesquisa centrada no termo exige que termos idênticos sejam pesquisados nos campos. Esta propriedade é chamada de <b>sincronicidade de campos</b>. Assim, uma fraqueza da pesquisa centrada no termo é a restrição de que os campos precisam ser pesquisados da mesma forma.


Com a pesquisa centrada no termo forçando todos os campos a se comportarem de forma idêntica, isso prejudica sua capacidade de usar a modelagem de signals? Realmente, esta estrégia prejudica a modelagem de signals.



#### 2. então vamos usar todos os campos com mesma análise

In [27]:
usersSearch = 'jornada nas estrelas patrick stewart william shatner'

df = search_solr(usersSearch, qf='title overview cast.name directors.name')
df_resumido = df[['Relevance Score', 'Movie Title']]
df_resumido.head(15)

Solr results


,Relevance Score,Movie Title
0,7.348200,Jornada nas Estrelas: Generations
1,6.559545,Jornada nas Estrelas V: A Última Fronteira
2,5.525932,Jornada nas Estrelas: O Filme
3,5.033387,X-Men: O Confronto Final
4,4.963281,Bill & Ted - Dois Loucos no Tempo
5,4.767649,Jornada nas Estrelas: Insurreição
6,4.767649,Jornada nas Estrelas: Nêmesis
7,4.727496,Máquina Quase Mortífera
8,4.712992,Osmose Jones
9,4.652231,Selvagem


In [28]:
df.head(15)

,Num,Relevance Score,Movie Title,Overview,Cast,Director
0,1,7.348200,Jornada nas Estrelas: Generations,"A viagem inaugural da terceira nave Enterprise está ameaçada pelo cientista louco Soren e apenas uma pessoa pode detê-lo, mas ela está morta há 78 anos.","[Patrick Stewart, Jonathan Frakes, Brent Spiner, LeVar Burton, Michael Dorn, Gates McFadden, Marina Sirtis, William Shatner, James Doohan, Walter Koenig, Malcolm McDowell, Alan Ruck, Whoopi Goldberg, Thomas Dekker, Cameron Oppenheimer, Jenette Goldstein, Tim Russ]",[David Carson]
1,2,6.559545,Jornada nas Estrelas V: A Última Fronteira,"Data estelar: 8454.130. O capitão Kirk, que está de férias, tem que enfrentar dois desafios: escalar o El Capitan, no Yosemite, e ensinar cantigas de acampamento para Spock. Mas o lazer é bruscamente interrompido quando um vulcaniano renegado rapta a Enterprise e a leva para desvendar os mais profundos segredos do universo.","[William Shatner, Leonard Nimoy, DeForest Kelley, James Doohan, Walter Koenig, Nichelle Nichols, George Takei, David Warner, Laurence Luckinbill, Charles Cooper, Cynthia Gouw, Todd Bryant, Spice Williams-Crosby, Rex Holman, George Murdock, Jonathan Simpson, Beverly Hart, Steve Susskind, Harve Bennett, Cynthia Blaise, Bill Quinn, Melanie Shatner, Michael Berryman]",[William Shatner]
2,3,5.525932,Jornada nas Estrelas: O Filme,"Um fenômeno alienígena de poder e tamanho descomunal se aproxima da Terra, destruindo tudo em seu caminho. A única nave que pode enfrentar esta força letal que ameaça a Terra é a U.S.S. Enterprise. O lendário comandante James T. Kirk (William Shatner) é convocado para a missão, mas um perigo não previsto pode destruir a U.S.S. Enterprise e toda a sua tripulação.","[William Shatner, Leonard Nimoy, DeForest Kelley, Stephen Collins, Persis Khambatta, James Doohan, Nichelle Nichols, George Takei, Walter Koenig, Majel Barrett, Grace Lee Whitney, Mark Lenard, Billy Van Zandt, Roger Aaron Brown, Gary Faga, Franklyn Seales]",[Robert Wise]
3,4,5.033387,X-Men: O Confronto Final,"É descoberta uma cura para os mutantes, que agora podem optar por manter seus poderes ou se tornarem seres humanos normais. A descoberta põe em campos opostos Magneto (Ian McKellen), que acredita que esta cura se tornará uma arma contra os mutantes, e os X-Men, liderados pelo professor Charles Xavier (Patrick Stewart).","[Hugh Jackman, Halle Berry, Ian McKellen, Patrick Stewart, Famke Janssen, Anna Paquin, Kelsey Grammer, James Marsden, Rebecca Romijn, Shawn Ashmore, Aaron Stanford, Vinnie Jones, Ellen Page, Daniel Cudmore, Ben Foster, Michael Murphy, Dania Ramirez, Shohreh Aghdashloo, Josef Sommer, Bill Duke, Eric Dane, Desiree Zurowski, Adrian Hough, Haley Ramm, Chris Claremont, Stan Lee, Cayden Boyd, Tanya Newbould, Anthony Heald, Cameron Bright, Connor Widdows, Kea Wong, Shauna Kain, Luke Pohl, Julian Richings, Lloyd Adams, Richard Yee, Via Saleaumua, Meiling Melançon, Omahyra Mota, Clayton Watmough, Ken Leung, Julian Christopher, Brad Kelly, Makenzie Vega Norfolk, Mi-Jung Lee, Benita Ha, Ron James, R. Lee Ermey, Lance Gibson, Aaron Pearl, Chelah Horsdal, John Pyper-Ferguson, Brenna O'Brien, Justin Callan, Alex Ferris, Peter Kawasaki, Ron Blecker, Emy Aneke, David Smith, Olivia Williams]",[Brett Ratner]
4,5,4.963281,Bill & Ted - Dois Loucos no Tempo,"Califórnia, 2691. Gênio do mal decide acabar com a duradoura paz mundial e resolve mandar exterminadores para acabar com a origem da ideologia pacifista então reinante: a dupla de roqueiros Bill e Ted, que deve ser morta e substituída por robôs.","[Keanu Reeves, Alex Winter, George Carlin, William Sadler, Joss Ackland, Pam Grier, Amy Stock-Poynton, Jim Martin, Hal Landon Jr., Annette Azcuy, Sarah Trigger, Chelcie Ross, Taj Mahal, Eleni Kelakos, Roy Brocksmith, J. Patrick McNamara, Dana Stevens, Carol Rosenthal, Chris Matheson, Brendan Ryan, William Thorne, Ed Gale, Arturo Gil, Tom Allard, Terry Finn, John Ehrin, Don Forney, Michael Chambers, Bruno Falcon, Ed Cambridge, Tad Horino, Wil

In [29]:
usersSearchElastic = 'jornada OR nas OR estrelas OR patrick OR stewart OR william OR shatner'
query = {
    'query': {
        'query_string': { 
            'query': usersSearchElastic,  #User's query
            'fields': ['title', 'overview', 'cast.name', 'directors.name'],  
            'minimum_should_match':'30%'
         }
    },
    'size': 50,
    'explain': True
}

df = search_elastic(usersSearch, query)
df_resumido = df[['Relevance Score', 'Movie Title']]
df_resumido.head(15)

Elasticsearch results


,Relevance Score,Movie Title
0,26.079668,Jornada nas Estrelas: Generations
1,22.645878,Jornada nas Estrelas: O Filme
2,22.251818,Jornada nas Estrelas V: A Última Fronteira
3,17.227531,Jornada nas Estrelas II: A Ira de Khan
4,17.055216,Jornada nas Estrelas: Insurreição
5,16.761562,Jornada nas Estrelas: Nêmesis
6,16.288609,Jornada nas Estrelas IV: A Volta para Casa
7,16.169870,Jornada nas Estrelas VI: A Terra Desconhecida
8,16.169870,Jornada nas Estrelas III: A procura de Spock
9,15.671905,Jornada nas Estrelas: Primeiro Contato


#### 3. outro exemplo: busca pelo ator Ben Affleck

In [30]:
usersSearch = 'ben affleck'

df = search_solr(usersSearch, qf='title overview cast.name directors.name')
df_resumido = df[['Relevance Score', 'Movie Title']]
df_resumido.head(15)

Solr results


,Relevance Score,Movie Title
0,5.946940,Medo da Verdade
1,5.946940,Argo
2,5.946940,Atração Perigosa
3,5.946940,A Lei da Noite
4,5.135891,Forças do Destino
5,4.844909,Amor Pleno
6,4.755284,Eu Ainda Estou Aqui
7,4.677504,Contato de Risco
8,4.570631,Procura-se Amy
9,4.551935,O Primeiro Milhão


In [31]:
df.head(15)

,Num,Relevance Score,Movie Title,Overview,Cast,Director
0,1,5.946940,Medo da Verdade,"Patrick Kenzie (Casey Affleck) e Angela Gennaro (Michelle Monaghan) são sócios em uma agência de detetives e também namorados. Um dia eles recebem a proposta de investigar o misterioso desaparecimento da pequena Amanda McCready (Madeline O'Brien), levada quando sua mãe, Helene (Amy Ryan), a deixou sozinha em casa. A polícia também investiga o caso, sendo liderada por Jack Doyle (Morgan Freeman). Patrick conhece bastante as pessoas do bairro em que Amanda vivia e passa a conversar com as pessoas. Logo descobre que a história que Helene contou à polícia não era verdadeira. Aos poucos ele percebe que há mais mentiras vindas dos envolvidos no caso.","[Casey Affleck, Michelle Monaghan, Morgan Freeman, Ed Harris, John Ashton, Amy Ryan, Amy Madigan, Titus Welliver, Michael Kenneth Williams, Edi Gathegi, Mark Margolis, Madeline O'Brien, Slaine, Trudi Goodman, Matthew Maher, Jill Quigg, Sean Malone, Brian Scannell, Jay Giannone, William Lee, William Marlowe, Daniel DeMiller Jr., Kenneth Butler Jr., Stephen Curran, Michael T. Blythe, Bob J. Leary, Mike Pusateri, Paul Sullivan, John McColgam, Nicholas Donovan, Joseph Thomas-O'Brien, Jimmy LeBlanc, Mary Bounphasaysonh, Fanshen Cox, Kippy Goldfarb, Elizabeth Duff, Cathie Callanan, Cameron Henry, Bobby Curcuro, Kevin Molis, Robert Wahlberg, Tom Kemp, Matt Podolske, Joseph Flaherty, Carla Antonino, Peg Holzemer, Chelsea Ladd, Josh Marchette, Tom McNeeley, Paul Hornung, Rena Maliszewski, Suzanne Schemm, Lonnie Farmer, Richard Snee, Dale Place, Gary Tanguay, Ted Reinstein, Celeste Oliva, Patrick Shea, Lewis D. Wheeler, Michele Proude, Tim Estiloz, Karen Scalia, John Belche, Raymond Alongi, Joey Vacchio, Eamon Brooks, Vincent H. Carolan, Frank G. Sullivan, Karen Ahern, Ellen Becker Gray, Shana Carr, Mark S. Cartier, Chemi Che-Mponda, Frank Durant, John Franchi, Mackenzie Hawe, Alex Milne, Nicholas Purcell, Alan Resnic]",[Ben Affleck]
1,2,5.946940,Argo,"Quando a revolução iraniana invade a embaixada dos EUA em Teerã, seis cidadãos americanos escapam. Agora, somente um homem é capaz de tirá-los de trás das linhas inimigas. Tony Mendez é um brilhante agente da CIA que se especializou em fugas impossíveis, mas seu mais recente plano é tão audacioso quando desesperador. Disfarçados como equipe de produção de um filme, Tony e os fugitivos devem se esconder às vistas de todos e o menor deslize pode significar morte instantânea.","[Ben Affleck, Alan Arkin, Bryan Cranston, John Goodman, Scoot McNairy, Tate Donovan, Christopher Denham, Kerry Bishé, Clea DuVall, Victor Garber, Rory Cochrane, Kyle Chandler, Chris Messina, Zeljko Ivanek, Titus Welliver, Richard Kind, Taylor Schilling, Keith Szarabajka, Bob Gunton, Page Leong, Omid Abtahi, Sheila Vand, Karina Logue, Ryan Ahern, Bill Tangradi, Jamie McShane, Roberto Garcia, Christopher Stanley, Jon Woodward Kirby, Victor McCay, Matt Nolan, J.R. Cacia, Bill Kalmenson, Rob Brownstein, David Sullivan, John Boyd, Yuriy Sardarov, Nikka Far, Aidan Sussman, Barry Livingston, Ali Saam, Araz Vahid Ebrahim Nia, Scott Anthony Leet, Michael Parks, Mehrdad Sarlak, Ray Haratian, Matthew Glave, Nancy Stelle, Mark Rhino Smith, Richard Dillane, Farshad Farahat, Alborz Basiratmand, Ruty Rutenberg, Michael Woolston, Sharareh Sedghi, Bobby Zegar, Ray Porter, Allegra Carpenter, Adrienne Barbeau, Houshang Touzie, Richard Allan Jones, Tim Quill]",[Ben Affleck]
2,3,5.946940,Atração Perigosa,"Doug MacRay (Ben Affleck) é muito bom na arte de planejar assaltos e lidera um grupo de ladrões de bancos que sempre sai impune, apesar das investigações do FBI. Um dia, ao realizar um assalto, seu parceiro Jem (Jeremy Renner) leva uma refém, por precaução. Ela é Claire Keesey (Rebecca Hall), subgerente do banco assaltado, solta próximo à praia algum tempo depois. O fato traumatiza Claire, deixando-a com sequelas. O problema é que Jem descobre que Claire mora a apenas quatro quarteirões do bando, tornando-se um

In [32]:
usersSearchElastic = 'Ben OR Affleck'
query = {
    'query': {
        'query_string': { 
            'query': usersSearchElastic,  #User's query
            'fields': ['title', 'overview', 'cast.name', 'directors.name'],  
            'minimum_should_match':'30%'
         }
    },
    'size': 50,
    'explain': True
}

df = search_elastic(usersSearch, query)
df_resumido = df[['Relevance Score', 'Movie Title']]
df_resumido.head(15)

Elasticsearch results


,Relevance Score,Movie Title
0,13.056545,Medo da Verdade
1,13.056545,Argo
2,13.056545,Atração Perigosa
3,13.056545,A Lei da Noite
4,11.298962,Forças do Destino
5,10.654331,Amor Pleno
6,10.442504,Eu Ainda Estou Aqui
7,10.290508,Contato de Risco
8,10.045986,Procura-se Amy
9,9.994625,O Primeiro Milhão


Neste exemplo, o primeiro resultado provavelmente não corresponderia à expectativa do usuário. Ben Afflect não atuou no filme "Medo da Verdade". As ferramentas estão priorizando esse resultado porque Ben Affleck foi o diretor do filme. Mas será que o usuário está considerando o diretor do filme?

<b>Este é mais um exemplo de discordância de signals. Apesar da pesquisa centrada no termo resolver o problema do elefante albino, esse tipo de pesquisa não resolve o problema de discordância de signals. </b>

### 4. então vamos tentar dar um boost no campo do elenco

In [33]:
usersSearch = 'ben affleck'

df = search_solr(usersSearch, qf='title overview cast.name^10 directors.name')
df_resumido = df[['Relevance Score', 'Movie Title']]
df.head(15)

Solr results


,Num,Relevance Score,Movie Title,Overview,Cast,Director
0,1,45.706306,Procura-se Amy,"Holden McNeil e Banky Edwards, criadores de uma famosa revista em quadrinhos em Nova Jérsia, vivem tranquilos, até Holden conhecer a bela roteirista Alyssa Jones. Apaixonado pela garota, sente-se traído ao descobrir suas preferências românticas. Decidem ser só bons amigos. Depois de uma proposta inusitada, Alyssa e Holden se separam e só se reencontram um ano depois...","[Ben Affleck, Joey Lauren Adams, Jason Lee, Dwight Ewell, Jason Mewes, Kevin Smith, Ethan Suplee, Scott Mosier, Casey Affleck, Matt Damon, Brian O'Halloran, Carmen Llywelyn, Guinevere Turner, Joe Quesada, Illeana Douglas, Welker White, Mike Allred]",[Kevin Smith]
1,2,45.291466,Amor Pleno,"Um homem (Ben Affleck), descontente com a sua vida, viaja a Paris e inicia uma profunda relação amorosa com uma europeia (Olga Kurylenko). Ele volta para os Estados Unidos e se casa com esta mulher, para ajudá-la a ter a permissão de estadia americana. Mas após o casamento, a relação dos dois se degrada. Neste momento, ele encontra uma antiga namorada (Rachel McAdams), com quem inicia um novo romance.","[Ben Affleck, Olga Kurylenko, Javier Bardem, Rachel McAdams, Tatiana Chiline, Romina Mondello, Tony O'Gans, Charles Baker]",[Terrence Malick]
2,3,41.272167,Aposta Máxima,"Ritchie (Justin Timberlake) luta para concluir sua universidade, mas tem problemas para arcar com as despesas do curso. Ele acaba entrando de cabeça no mundo das apostas online de pôquer. Antes ele apenas apresenta o jogo para outras pessoas, mas também acaba jogando para conseguir dinheiro. Ele perde tudo, mas está convencido de que foi passado para trás. É aí que decide visitar a Costa Rica e pressionar o executivo Ivan Block (Ben Affleck). Impressionado com o jovem, o empresário acaba acolhendo Ritchie em seu negócio.","[Ben Affleck, Gemma Arterton, Justin Timberlake, Anthony Mackie, David Costabile, John Heard, Michael Esper, Oliver Cooper, Christian George, Yul Vazquez, James Molina, Louis Lombardi, Vincent Laresca, Sam Palladio, Ben Schwartz, Dayo Okeniyi, Laura Aleman, Sam Upton, Hugh Scott]",[Brad Furman]
3,4,40.678707,Código de Honra,"Vindo de uma família humilde, o jovem David Greene (Bredan Fraser) conseguiu uma bolsa em uma grande e tradicional escola. Lá, ele se torna a estrela do time de futebol americano do colégio, gerando a admiração e a inveja de muitos de seus colegas. Mas tudo muda quando uma verdade vem a tona: David é judeu. A partir daí, os preconceitos ficam cada vez mais visíveis.","[Brendan Fraser, Matt Damon, Chris O'Donnell, Randall Batinkoff, Ben Affleck, Amy Locane, Cole Hauser, Anthony Rapp, Zeljko Ivanek, Kevin Tighe, Ed Lauter, Peter Donat, Stanton Denman, John Cunningham]",[Robert Mandel]
4,5,40.336370,Fora de Controle,"Um simples acidente de carro transforma dois homens aparentemente normais em inimigos mortais. A confusão ocorre porque, atrasado para uma reunião no tribunal, o advogado Gavin Banek (Ben Affleck) sai costurando pelo trânsito e, na outra pista, vem Doyle Gipson (Samuel L. Jackson), um pai que está lutando pelo direito de ver seus filhos, algo que depende da decisão de um juiz que não tem nenhum tempo a perder. Aparentemente, Banek e Gipson são bem diferentes. Porém, no meio do caminho dos dois, um simples acidente prova que a raiva pode empatar qualquer disputa e que os meios pelos quais eles tentarão arruinar um ao outro podem ser diferentes, mas seus objetivos serão sempre os mesmo: cada um deles tentará, sistematicamente e incansavelmente, destruir a vida do outro por causa daquilo que perderam.","[Ben Affleck, Samuel L. Jackson, Kim Staunton, Toni Collette, Sydney Pollack, Matt Malloy, William Hurt, John Benjamin Hickey, Amanda Peet, Michael Pitt, Myra Lucretia Taylor, Richard Jenkins, Dylan Baker]",[Roger Michell]
5,6,39.999740,Menina dos Olhos,"Ollie Trinke (Ben Affleck) é um homem casado e bem-sucedido no emprego, que parece ter tudo o que sempre quis na vida. Porém su

Mas e se...

In [34]:
usersSearch = 'woody allen'

df = search_solr(usersSearch, qf='title overview cast.name^10 directors.name')
df_resumido = df[['Relevance Score', 'Movie Title']]
df.head(15)

Solr results


,Num,Relevance Score,Movie Title,Overview,Cast,Director
0,1,43.267560,Amante a Domicílio,"Murray (Woody Allen) é um senhor de idade que, durante uma conversa sobre sexo, diz que conhece um gigolô. Ao saber o quanto poderia ganhar como cafetão, ele tenta convencer seu amigo, Fioravante (John Turturro), a entrar para o ramo. Só que ele é um pacato jardineiro e não quer se envolver em algo do tipo. Após muita insistência de Murray, Fioravanti topa fazer um programa com a doutora Parker (Sharon Stone), que está bastante insegura por ser a primeira vez que trai o marido. O sucesso do encontro faz com que a fama de Fioravanti corra entre as amigas da doutora, assim como ele mesmo passa a notar melhor as qualidades da nova profissão.","[John Turturro, Woody Allen, Vanessa Paradis, Liev Schreiber, Sharon Stone, Sofía Vergara, Loan Chabanol, Bob Balaban, Max Casella, Aida Turturro, Eugenia Kuzmina, Ted Sutherland, Dante Hoagland, David Margulies, Abe Altman, Sol Frieder, Michael Badalucco, Allen Lewis Rickman]",[John Turturro]
1,2,41.971940,Memórias,"Sandy Bates (Allen), um lendário cineasta famoso por suas comédias, está cansado de ser engraçado. Em um fim de semana, à beira de um ataque de nervos, Bates comparece a uma retrospectiva de seus filmes, onde acaba tendo que se confrontar com o significado de seu trabalho, com as lembranças de seu grande amor, Dorrie (Charlotte Rampling), e com os méritos de um relacionamento sério com sua nova namorada, Isobel (Marie-Christine Barrault). Atormentado por alucinações, por estranhas visitas e pelos executivos impiedosos dos estúdios tentando reeditar seu novo filme, Bates luta para encontrar um motivo para continuar vivendo.","[Woody Allen, Charlotte Rampling, Jessica Harper, Marie-Christine Barrault, Tony Roberts, Daniel Stern, Amy Wright, Helen Hanft, John Rothman, Sharon Stone, Eli Mintz, Gabriel Barre, Judith Roberts]",[Woody Allen]
2,3,41.615715,Maridos e Esposas,"Gabe e sua mulher Judy ficam chocados ao saber que seus melhores amigos Sally e Jack estão se separando e começam a questionar o próprio relacionamento. Enquanto Gabe pensa em flertar com uma de suas alunas da faculdade, Sally e Jack descobrem que a vida de solteiro não é assim tão boa e pensam em reatar. (e 16 - Estimado 16 Anos)","[Woody Allen, Mia Farrow, Judy Davis, Sydney Pollack, Juliette Lewis, Liam Neeson, Lysette Anthony, Cristi Conaway, Timothy Jerome, Ron Rifkin, Blythe Danner, Brian McConnachie, Caroline Aaron, Galaxy Craze]",[Woody Allen]
3,4,41.265490,Sonhos de um Sedutor,"Um neurótico crítico de cinema (Woody Allen), obcecado pelo filme Casablanca (1942), é abandonado por sua esposa e seu ego fica destruído. Seu exemplo de herói é o cara durão interpretado por Humphrey Bogart em muitos de seus filmes. Então Bogart começa a aparecer ao vivo para lhe dar conselhos. Com o encorajamento de um casal de amigos ele realmente tenta namorar novamente, mas com resultados pouco satisfatórios. (e 14 - Estimado 14 Anos)","[Woody Allen, Diane Keaton, Tony Roberts, Jerry Lacy, Susan Anspach, Jennifer Salt, Joy Bang, Viva, Ted Markland, Michael Greene, Susanne Zenor, Diana Davila, Mari Fletcher, Mark Goddard, Tom Bullock]",[Herbert Ross]
4,5,41.265490,Poderosa Afrodite,"Em Nova York, um casal adota um menino e com o tempo o pai adotivo (Woody Allen) decide saber quem é a mãe biológica de seu filho. Ele descobre que ela é uma prostituta chamada Linda (Mira Sorvino), que em filmes pornográficos usa o nome Judy Cum e que nem sabe quem é o pai do garoto. O pai adotivo decide então aconselhá-la a abandonar este tipo de vida.","[Woody Allen, Mira Sorvino, Helena Bonham Carter, F. Murray Abraham, Donald Symington, Claire Bloom, Olympia Dukakis, Michael Rapaport, David Ogden Stiers, Jack Warden, Peter Weller, Danielle Ferland, Paul Giamatti]",[Woody Allen]
5,6,41.265490,Sonhos Eróticos de Uma Noite de Verão,"No início do século XX, três casais se reúnem em uma casa de campo. Os anfitriões têm problemas de ordem sexual, um outro casal é

Aqui claramente a pesquisa não dá certo. A busca provavelmente é por um diretor, mas o sistema prioriza o filme em que ele atuou, que não é algo muito comum (privilegia a raridade).

Assim, o boost, em vez de atacar diretamente o problema das frequências dos termos, ele força uma pontuação. E se o usuário tivesse procurado um diretor que apenas uma vez aparecesse como membro do elenco? Essa solução aumenta a pontuação do filme em que o diretor participou do elendo e empurra para baixo os filmes de direção desse diretor. Não é o que os usuários esperam.

<b>A solução passa por manipular as frequências dos termos.</b>

# Conclusão
![title](img/bicho.jpg)

<b><center>Se correr o bicho pega, se ficar o bicho come... </center></b>

Mas há luz no fim do túnel!

Pesquisas centradas no campo e pesquisas centradas no termo formam um yin-yang: ambos tem vantagens e desvantagens.
A solução é usar um pouco de cada um. 